In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import os

In [2]:
# Define the path to the zip file and the directory to extract to
zip_file_path = 'Plant_leaf_diseases_dataset_with_augmentation.zip'
extract_dir = 'Plant_leave_diseases_dataset_with_augmentation'

class_names = sorted(os.listdir(extract_dir))

In [3]:
def process_path(file_path):
    # Get the label from the directory name
    parts = tf.strings.split(file_path, os.path.sep)
    label = parts[-2]
    label = tf.where(class_names == label)[0][0]
    
    # Load and preprocess the image
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0  # Normalize to [0,1] range
    
    return img, label

In [4]:
# List the classes
class_names = sorted(os.listdir(extract_dir))

# Create a dataset of file paths
list_ds = tf.data.Dataset.list_files(f'{extract_dir}/*/*', shuffle=True)

# Split into train and validation sets
val_size = int(0.2 * len(list(list_ds)))
train_size = len(list(list_ds)) - val_size

train_ds = list_ds.take(train_size)
val_ds = list_ds.skip(train_size).take(val_size)

# Map the process_path function to the datasets
train_ds = train_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

# Print the number of class names
num_classes = len(class_names)
print("Number of classes:", num_classes)
print("Class names:", class_names)

Number of classes: 39
Class names: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Background_without_leaves', 'Blueberry___healthy', 'Cherry___Powdery_mildew', 'Cherry___healthy', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___Northern_Leaf_Blight', 'Corn___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot'

In [5]:
batch_size = 32

# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomBrightness(0.2),
])

# Apply data augmentation to the training dataset
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.cache().shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

In [6]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)  # Add Dropout layer for regularization
predictions = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=50,
                  )

Epoch 1/50
1538/1538 [==============================] - 96s 57ms/step - loss: 3.4413 - accuracy: 0.1235 - val_loss: 2.2538 - val_accuracy: 0.4122
Epoch 2/50
1538/1538 [==============================] - 51s 33ms/step - loss: 3.2695 - accuracy: 0.1549 - val_loss: 1.6939 - val_accuracy: 0.5475
Epoch 3/50
 146/1538 [=>............................] - ETA: 34s - loss: 3.2100 - accuracy: 0.1637

KeyboardInterrupt: 

In [ ]:
# Fine-tuning: Unfreeze some layers and retrain
for layer in base_model.layers[-20:]:  # Unfreeze the last 20 layers
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model again with the unfrozen layers
history_fine = model.fit(train_ds,
                         validation_data=val_ds,
                         epochs=20,
                         callbacks=[reduce_lr, early_stopping])

Epoch 1/20
1538/1538 [==============================] - 59s 36ms/step - loss: 3.2277 - accuracy: 0.1728 - val_loss: 1.5587 - val_accuracy: 0.5768 - lr: 1.0000e-05
Epoch 2/20
1538/1538 [==============================] - 48s 31ms/step - loss: 2.8748 - accuracy: 0.2268 - val_loss: 1.5501 - val_accuracy: 0.5631 - lr: 1.0000e-05
Epoch 3/20
1538/1538 [==============================] - 50s 33ms/step - loss: 2.7855 - accuracy: 0.2461 - val_loss: 1.4776 - val_accuracy: 0.5882 - lr: 1.0000e-05
Epoch 4/20
1538/1538 [==============================] - 51s 33ms/step - loss: 2.7312 - accuracy: 0.2588 - val_loss: 1.4038 - val_accuracy: 0.6050 - lr: 1.0000e-05
Epoch 5/20
1538/1538 [==============================] - 50s 32ms/step - loss: 2.6963 - accuracy: 0.2685 - val_loss: 1.3578 - val_accuracy: 0.6223 - lr: 1.0000e-05
Epoch 6/20
1538/1538 [==============================] - 50s 32ms/step - loss: 2.6572 - accuracy: 0.2794 - val_loss: 1.3402 - val_accuracy: 0.6348 - lr: 1.0000e-05
Epoch 7/20
1538/1538 [